In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [41]:
a = torch.randn(2, 5)
b = np.random.randn(2, 5)
print(a.shape, b.shape, sep=' --- ')
print(a.shape[0], b.shape[0], sep=' --- ')

torch.Size([2, 5]) --- (2, 5)
2 --- 2


In [3]:

# Create a simple dataset
class LinearRegressionDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.X = torch.randn(num_samples, 5)
        self.y = torch.sum(self.X, dim=1, keepdim=True) + torch.randn(num_samples, 1) * 0.1

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define the model
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(5, 1)

    def forward(self, x):
        return self.linear(x)


In [19]:
# Create datasets and dataloaders
train_dataset = LinearRegressionDataset(num_samples=800)
test_dataset = LinearRegressionDataset(num_samples=200)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)


In [32]:
train_dataset.X, train_dataset.y
import pandas as pd
import plotly.express as px

# Convert tensors to pandas DataFrame for plotting
df = pd.DataFrame(train_dataset.X.numpy(), columns=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5'])
df['Target'] = train_dataset.y.numpy().flatten()

# Plotting
fig = px.scatter(df, x='Feature1', y='Target', title='Feature 1 vs Target')
fig.show()



In [21]:
# Initialize the model, loss function, and optimizer
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [22]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        test_loss += criterion(outputs, batch_y).item()
    
    avg_test_loss = test_loss / len(test_loader)
    print(f'Average test loss: {avg_test_loss:.4f}')

# Make predictions on new data
new_data = torch.randn(5, 5)  # 5 samples with 5 features each
predictions = model(new_data)
print("Predictions for new data:")
print(predictions)

Epoch [10/100], Loss: 0.0043
Epoch [20/100], Loss: 0.0024
Epoch [30/100], Loss: 0.0105
Epoch [40/100], Loss: 0.0114
Epoch [50/100], Loss: 0.0050
Epoch [60/100], Loss: 0.0085
Epoch [70/100], Loss: 0.0132
Epoch [80/100], Loss: 0.0186
Epoch [90/100], Loss: 0.0169
Epoch [100/100], Loss: 0.0028
Average test loss: 0.0103
Predictions for new data:
tensor([[ 1.3996],
        [ 0.6587],
        [ 1.1023],
        [-1.2221],
        [-3.1154]], grad_fn=<AddmmBackward0>)
